# Graph Execution
本文档介绍了如何使用机器学习按品种对鸢尾花进行分类（归类）。本文档深入介绍了如何使用 TensorFlow 代码执行此项任务，以此来解释机器学习的基本原理。

In [5]:
#版权归https://www.tensorflow.org/get_started/get_started_for_beginners
from __future__ import  absolute_import,division,print_function

import os
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
#import tensorflow.contrib.eager as tfe
import argparse
#tf.enable_eager_execution()

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', default=100, type=int, help='batch size')
parser.add_argument('--train_steps', default=1000, type=int,
                    help='number of training steps')

print("tensorflow version:{}".format(tf.VERSION))
print("Eager Excution:{}".format(tf.executing_eagerly()))

D:\Anaconda\envs\python35\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


tensorflow version:0.12.0-rc1


AttributeError: module 'tensorflow' has no attribute 'executing_eagerly'

# 导入和解析数据集

In [2]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']

SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def load_data(label_name = 'Species'):
    #训练集提取
    train_path = tf.keras.utils.get_file(fname= os.path.basename(TRAIN_URL),
                                        origin = TRAIN_URL)
    train = pd.read_csv(filepath_or_buffer=train_path,names=CSV_COLUMN_NAMES,header=0)
   
    train_features,train_label = train,train.pop(label_name)
    
    #测试集提取
    test_path = tf.keras.utils.get_file(fname=os.path.basename(TEST_URL),
                                       origin = TEST_URL)
    test = pd.read_csv(filepath_or_buffer=test_path,names=CSV_COLUMN_NAMES,header=0)
    
    test_features,test_label = test,test.pop(label_name)
    
    return(train_features,train_label),(test_features,test_label)

In [3]:
(train_x,train_y),(test_x,test_y) = load_data()
print(train_x[:5])

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3


# 描述数据/特征列
特征列是一种数据结构，告知模型如何解读每个特征中的数据。

In [4]:
#为所有特征创造一个特征列
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

# 选择模型类型

In [5]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10,10],
    n_classes = 3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\THINK\\AppData\\Local\\Temp\\tmps3jxrvsz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D1CD449780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# 训练模型

In [6]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset.make_one_shot_iterator().get_next()

In [7]:
classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y, 1000),
    steps= 50)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\THINK\AppData\Local\Temp\tmps3jxrvsz\model.ckpt.
INFO:tensorflow:loss = 2162.4663, step = 1
INFO:tensorflow:Saving checkpoints for 50 into C:\Users\THINK\AppData\Local\Temp\tmps3jxrvsz\model.ckpt.
INFO:tensorflow:Loss for final step: 240.00427.


# 评估模型

In [8]:
def eval_input_fn(features, labels=None, batch_size=None):
    """An input function for evaluation or prediction"""
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (dict(features), labels)

    # Convert inputs to a tf.dataset object.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

In [9]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, 1000))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-08-02:42:35
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\THINK\AppData\Local\Temp\tmps3jxrvsz\model.ckpt-50
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-08-02:42:35
INFO:tensorflow:Saving dict for global step 50: accuracy = 0.8666667, average_loss = 0.31349114, global_step = 50, loss = 9.404734
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 50: C:\Users\THINK\AppData\Local\Temp\tmps3jxrvsz\model.ckpt-50

Test set accuracy: 0.867



# 预测

In [10]:
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}
expected = ['Setosa', 'Versicolor', 'Virginica']


In [11]:
predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(predict_x,
                                  labels=None,
                                  batch_size=1000))


In [12]:
for pred_dict, expec in zip(predictions, expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    print(template.format(SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\THINK\AppData\Local\Temp\tmps3jxrvsz\model.ckpt-50
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (95.6%), expected "Setosa"

Prediction is "Versicolor" (57.3%), expected "Versicolor"

Prediction is "Virginica" (80.2%), expected "Virginica"


# 疑问

In [13]:
#lambda函数
#func = lambda 参数：表达式 
#注意返回的是函数不是值，所以值应该用func()表示
x = 1
y = 2
a = lambda :x+y
print(a())

3


In [14]:
# {'logits': array([ 3.244726  ,  0.00367624, -2.9622233 ], dtype=float32), 'probabilitie
#  s': array([0.9604873 , 0.03757696, 0.00193574], dtype=float32), 'class_ids': array([0],
#  dtype=int64), 'classes': array([b'0'], dtype=object)}